In [1]:
#import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow_hub as hub
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# ImageDataGenerator is used for image augmentation and rescaling

train_datagen = ImageDataGenerator(
                rescale=1/255.0,
                rotation_range=35,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest',
                #validation_split=0.2
                )

test_datagen = ImageDataGenerator(rescale=1/255.0, 
                                  #validation_split=0.2
                                  )

In [3]:
#Resize Image

img_width = 224
img_height = 224

In [4]:
#Read training data using flow_from_directory

train_data_generator = train_datagen.flow_from_directory(directory=r'C:\Users\Hello\Desktop\Image_Manipulation_Detection\data\data\traindev',
                                                   target_size = (img_width, img_height),
                                                   class_mode = 'categorical',
                                                   batch_size = 16,
                                                   #subset = 'training'
                                                   )

Found 7470 images belonging to 3 classes.


In [5]:
#Read test data using flow_from_directory

validation_data_generator = test_datagen.flow_from_directory(directory=r'C:\Users\Hello\Desktop\Image_Manipulation_Detection\data\data\test',
                                                   target_size = (img_width, img_height),
                                                   class_mode = 'categorical',
                                                   batch_size = 16,
                                                   shuffle=False,
                                                   #subset = 'validation'
                                                   )

Found 830 images belonging to 3 classes.


In [6]:
# Setup input shape to the model
INPUT_SHAPE = [None, img_width, img_height, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = 3 # number of unique labels

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

In [7]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), 
      optimizer=tf.keras.optimizers.Adam(), 
      metrics=["accuracy"] 
  )

  # Build the model
  model.build(INPUT_SHAPE) 

  return model

In [8]:
# Create a model and check its details
model = create_model()
model.summary()

Building model with: https://tfhub.dev/google/imagenet/inception_v3/classification/5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  23853833  
_________________________________________________________________
dense (Dense)                multiple                  3006      
Total params: 23,856,839
Trainable params: 3,006
Non-trainable params: 23,853,833
_________________________________________________________________


In [9]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_acc",
                                                  patience=3) # stops after 3 rounds of no improvements

In [10]:
# How many rounds should we get the model to look through the data?
NUM_EPOCHS = 100

In [11]:
# Modify the train_model() function to return history
def train_model():
    """
    Trains a given model and returns the trained version.
    """
    # Create a model
    model = create_model()

    # Fit the model to the data passing it the callbacks we created
    history = model.fit(
        x=train_data_generator,
        epochs=NUM_EPOCHS,
        validation_data=validation_data_generator,
        validation_freq=1,  # check validation metrics every epoch
        callbacks=[early_stopping]
    )

    return model, history


In [ ]:
# Train the model and obtain the trained model and history
model, history = train_model()

In [ ]:
# Training accuracy history
history.history['accuracy']

In [ ]:
# Validation accuracy history
history.history['val_accuracy']

In [ ]:
# Access the training and validation accuracy from the history object
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training and validation accuracy
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Get the class labels
class_labels = list(validation_data_generator.class_indices.keys())

# Print the class labels
print(class_labels)

In [ ]:
# Get the total number of samples in the validation dataset
num_samples = len(validation_data_generator.filenames)

# Get the batch size used in the generator
batch_size = validation_data_generator.batch_size

# Calculate the number of steps required to iterate through the entire dataset
num_steps = int(np.ceil(num_samples / batch_size))

# Initialize empty arrays to store the images and labels
x_test = []
y_true = []

# Iterate through the generator to obtain the images and labels
for _ in range(num_steps):
    images, labels = validation_data_generator.next()
    x_test.extend(images)
    y_true.extend(np.argmax(labels, axis=1))

# Convert the arrays to NumPy arrays
x_test = np.array(x_test)
y_true = np.array(y_true)

In [ ]:
# Get the predictions and convert predictions to class labels
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1) 

In [ ]:
# Load the confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Get the class labels
class_labels = list(validation_data_generator.class_indices.keys())

# Display the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
#Print Precision, Recall, F-score
print(classification_report(validation_data_generator.classes[validation_data_generator.index_array], y_pred))

In [ ]:
# Get a batch of test images and their corresponding labels
images, true_labels = validation_data_generator.next()

# Convert the predicted probabilities to class labels
predicted_labels = [class_labels[np.argmax(pred)] for pred in predictions]

# Display the images with their labels
num_samples = 5  # Number of samples to display
fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))

for i in range(num_samples):
    # Display the image
    axes[i].imshow(images[i])
    axes[i].axis('off')
    
    # Set the title with actual and predicted labels
    actual_label = class_labels[np.argmax(true_labels[i])]
    predicted_label = predicted_labels[i]
    title = f"Actual: {actual_label}\nPredicted: {predicted_label}"
    axes[i].set_title(title)

plt.tight_layout()
plt.show()